In [ ]:
###.... This is an integrated snop program that outputs c4s.csv and supply.csv from a set of inputs.
###.... More details can be found in README.MD

#********************************************************************
# Notes:

# 1) FourthShift Auto-updates happen everyday at 6:00 PM GMT 
#_____________________________________________________________________

import pandas as pd
import numpy as np
import calendar

import time
import datetime
from datetime import date, timedelta

from xlrd import open_workbook

pd.options.mode.chained_assignment = None  # Disable pesky chained_assignment warnings

start = time.time()

#**************************************************************
#                 Flags for manipulation
#______________________________________________________________

# Done in the first week of every month
no_india_orders = False    # If India does'nt have any enough orders (month beginning)

remove_outliers = True # Flag for removing margin outliers
negative_outlier = -0.25
positive_outlier = 2

use_local_file = True    # For Krishnan's file

local_order_file = r'\FS_Orders.xls'
local_bom_file = r'\FS_BILL.xlsx'


# path to store output
localpath = '.'
# localpath = r"C:\Users\kashir\Documents\SnOP\00_Model"

#**************************************************************
#                 Date conversion functions
#______________________________________________________________

def get_date(s_year, s_month):
    '''Function to get date from year and month
       Arg: 
         (s_year, s_month as <string, int>)
       Returns: 
         date as datetime.date'''
    dt = pd.to_datetime(int(s_year)*10000+int(s_month)*100+1, format = "%Y%m%d")
    return dt.date()

def get_last_day(idt):
    '''Function to get the last date of a month
       Arg: 
          idt as date
       Returns:
          last date as datetime.date
       '''
    dt = pd.to_datetime(idt) + pd.tseries.offsets.MonthEnd(1)
    return dt.date()

#*********************************************************************
#                      Variables and File Paths
#_____________________________________________________________________

# SnOP Horizon in months
snop_horizon = 15

# Last day of this month
thismonthend = (datetime.datetime.now()+pd.tseries.offsets.MonthEnd(1)).date()

# Site ID standardization
siteID_dict = {"IN": "ID", "BA": "IN", "J1": "HU", 
               "DB": "AE", "GR": "US", "HU": "HU", 
               "TH": "TH", "SG": "SG", "PA": "PA", 
               "BR": "PA"}

# Basic columns
base_cols = ['DataType', 'Site', 'ProductID', 'ProductDesc', 
             'CustomerNo', 'CustName', 'CustCountry', 'AM',  
             'Order', 'By', 'Qty', 'GBPValue']

#... File Paths ...
#..................

# Local files
# localpath = r"C:\Users\kashir\Documents\SnOP\00_Model"

# Live files
livecognospath = r"\\10.5.20.5\00_Model\Cognos"
livefspath = r"\\10.5.20.5\00_Model\FS\FSLive"

# Cognos and FouthShift files
if use_local_file is True:
    fs_orders_file = localpath + local_order_file
    fs_bom_file = localpath + local_bom_file
    cis_file = localpath + r'\CIS.txt'
    volume_file = localpath + r'\Volume.txt'
else:
    fs_orders_file = livefspath + r'\FS_Orders.xls'
    fs_bom_file = livefspath + r'\FS_BILL.xlsx'
    cis_file = livecognospath + r'\CIS_a.txt'
    volume_file = livecognospath + r'\Volume_a.txt'

# Actuals file (from Darshil)
actuals_file = localpath + r'\Actuals.xlsx'

# Brazil file (from Luiz / Diego)
brazil_file = localpath + r'\BRAZIL.xlsx'

# F1, F2, F3
f1_file = localpath + r'\F1.xlsx'
f2_file = localpath + r'\F2.xlsx'
f3_file = localpath + r'\F3.xlsx'
plan_file = localpath + r'\Plan.xlsx'

# Lookup file
lookups_file = localpath + r'\SnOP Lookups.xlsx'
lookups_xls = pd.ExcelFile(lookups_file)

#... File-based DataFrames ...
#.............................

cognos_cols = 'DataType,Combi,Month,Year,Qty_Val'.split(',')

df_cis = pd.read_csv(cis_file, sep=None, encoding = "UTF-16", 
                        header=None, engine='python', names = cognos_cols, usecols=[0, 1, 3, 4, 5])
df_volume =  pd.read_csv(volume_file, sep=None, encoding = "UTF-16", 
                        header=None, engine='python', names = cognos_cols, usecols=[0, 1, 3, 4, 5])

df_orders = pd.read_excel(fs_orders_file, header=0)
df_orders.PromDock_InvoiceDate = pd.to_datetime(df_orders.PromDock_InvoiceDate).dt.date

df_bom = pd.read_excel(fs_bom_file, header=0)
df_bom = df_bom.drop(axis=0, index=0) # drop the first row
df_bom = df_bom.drop(columns='IsBR')

df_actuals = pd.read_excel(actuals_file, 'Actuals', skiprows=4, header=0)
df_actuals.By = pd.to_datetime(df_actuals.By).dt.date

df_brazil = pd.read_excel(brazil_file, header=0)
df_brazil.By = pd.to_datetime(df_brazil.By).dt.date

# F1, F2, F3, ...
df_f1 = pd.read_excel(f1_file, 'F1', skiprows=4, header=0)
df_f2 = pd.read_excel(f2_file, 'F2', skiprows=4, header=0)
df_f3 = pd.read_excel(f3_file, 'F3', skiprows=4, header=0)
df_plan19 = pd.read_excel(plan_file, skiprows=4, header=0)

#... Lookups DataFrame ...
#.......................
df_ryo_pop = pd.read_excel(lookups_xls, 'RYOPOP', header=0)
# df_divadj = pd.read_excel(lookups_xls, '2018DivAdjust', header=0)
df_am = pd.read_excel(lookups_xls, 'AM_names', header=0).drop_duplicates('AM Code')
df_custgrp = pd.read_excel(lookups_xls, 'Customer_names', 
                           header=0).drop_duplicates('CustName')

# df_region = pd.read_excel(lookups_xls, 'CountryRef', header=0).drop_duplicates('Country')
df_prodref = pd.read_excel(lookups_xls, 'ProductRef', header=0, skiprows=4)
df_prodref = df_prodref.rename(columns = {'S1': 'Seg1', 
                                            'S2': 'Seg2', 'S3': 'Seg3', 'S4': 'Seg4' })
df_gamechangers = pd.read_excel(lookups_xls, 'GameChangers', header=0, skiprows=4, usecols=[1,2])

df_exceptions = pd.read_excel(lookups_xls, 'Master', skiprows=4, header=0, usecols = 'Y').drop_duplicates('Logic')
df_mfgsitechange = pd.read_excel(lookups_xls, 'MfgSite', header=4)

df_machine = pd.read_excel(lookups_xls, sheet_name='MachineRef', header=0, skiprows=4, 
                           converters={'MachineID': str})
df_machinemap = pd.read_excel(lookups_xls, sheet_name='MachineMap', header=0, skiprows=4, 
                              converters={'MachineID': str}, usecols=[0,1,2,3,4,5])

df_sites = pd.read_excel(lookups_xls, sheet_name='Master', header=0, skiprows=4, usecols="O:P").dropna()
df_holidays = pd.read_excel(lookups_xls, sheet_name='Master', header=0, skiprows=4, usecols="Q:R")

df_periods = pd.read_excel(lookups_xls, sheet_name='Master', header=0, skiprows=4, usecols="AA:AC")
df_periods = df_periods.dropna()

# Generate EssBy from Year_Period
df_periods['EssBy'] = [get_last_day(get_date(x, y)) for x, y in df_periods.Year_Period.str.split('-').tolist()]
df_periods.iloc[:, 1:4] = df_periods.iloc[:, 1:4].apply(pd.to_datetime, errors='coerce') # convert to datetime
df_periods.iloc[:, 1:4] = df_periods.iloc[:, 1:4].apply(lambda x: x.dt.date) # convert to datetime.date

# Get the start date as the minimum of Year_Period
start_date = pd.to_datetime(df_periods.Start.min()).date()

years = [start_date.year, start_date.year+2] #isin requires array

# Generate dictionary for mapping periods to Year_Period based on Essentra Calendar
df_essdict = df_periods.set_index('Year_Period')['EssBy'].to_dict()

def get_essby(df, start_date=start_date):
    '''Args: 
         df as pd dataframe, <start_date> defaulted from minum of df_periods.Start date
       Output:
         df with EssBy column'''
    
    df.By = pd.to_datetime(df.By).dt.date

    # Select only from the minimum start period if given
    if start_date != None:
        df = df.loc[df.By >= start_date, :].reset_index(drop=True)

    df['EssBy'] = [k for x in df.By 
                           for i, j, k in zip(df_periods.Start, df_periods.End, df_periods.EssBy) 
                           if i <= x <= j]
    return df

# %%time
#*********************************************************************
#                            Cognos Cleanup     
#_____________________________________________________________________

#...   Get the Forecast ...
#..........................

# Filter Forecast
forecastfilter = (((df_cis.DataType == 'Actual / Forecast') | (df_cis.DataType == 'Value GBP')) &
                (df_cis.Year.isin(years)) & \
                (df_cis.Month.isin(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])) & \
                (df_cis.Combi != 'TOTAL') & \
                (df_cis.Qty_Val != 0))

df0_forecast = df_cis[forecastfilter]

# Pivot the values to get Forecast, Month and Year Values
df1_forecast = df0_forecast.pivot_table(index = ['Combi', 'Month', 'Year'],
                           columns = 'DataType',
                           values = 'Qty_Val',
                           aggfunc = 'sum')

# Reset the index and show the values
df1_forecast = df1_forecast.reset_index().rename_axis(None).rename_axis(None, axis=1)

# Add new / Rename columns in line with SnOP Base Model
df1_forecast['DataType'] = 'Forecast'
df1_forecast = df1_forecast.rename(columns = {'Value GBP': 'GBPValue',
                      'Actual / Forecast': 'Qty'})

# Convert Months to MonthNum to get the last date of the month
df1_forecast['MonthNum'] = df1_forecast.Month.apply(lambda x: dict((v, k) for k, v in enumerate(calendar.month_abbr))[x])
df1_forecast['By'] = pd.to_datetime(df1_forecast.Year*10000 + 
                                    df1_forecast.MonthNum*100+1, format = "%Y%m%d") + pd.tseries.offsets.MonthEnd(1)

#...   Get the Plan   ...
#........................

# Filter Budget (Plan)
planfilter = (((df_cis.DataType == 'Budget') | (df_cis.DataType == 'Budget Value GBP')) &
                (df_cis.Year.isin(years)) & \
                (df_cis.Month.isin(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', \
                                     'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])) & \
                (df_cis.Combi != 'TOTAL') & \
                (df_cis.Qty_Val != 0))

df_plan = df_cis[planfilter]

# Pivot to get sum of Plan, Month and Year Values
df1_plan = df_plan.pivot_table(index = ['Combi', 'Month', 'Year'],
                           columns = 'DataType',
                           values = 'Qty_Val',
                           aggfunc = 'sum').reset_index().rename_axis(None).rename_axis(None, axis=1)

# Set up columns with correct names
df1_plan['DataType'] = 'Plan'
df1_plan = df1_plan.rename(columns = {'Budget':'Qty', 'Budget Value GBP':'GBPValue' })

# Multiply the budget value with 1,000
df1_plan.GBPValue = df1_plan.GBPValue * 1000

# Convert Months to MonthNum
df1_plan['MonthNum'] = df1_plan.Month.apply(lambda x: dict((v, k) for k, v in enumerate(calendar.month_abbr))[x])


# Remove 2019 of Cognos from data from df1_plan, because this is coming from Jared + Darshil's FDS file
df1_plan = df1_plan[df1_plan.Year == 2018].reset_index(drop=True)

#... Get the details from Volume file ...
#........................................

# Filter the Volume file
volumefilter = (df_volume.Month.isin(['Prod Name', 'Cust Name', 'Cust Country', \
                                      'Item ID', 'AM'])) & (df_volume.Year.isin(years))

df1_volume = df_volume[volumefilter]

# Pivot and split out items into columns from rows in 'Month' column
df1_volume = df1_volume.pivot_table(index=['DataType', 'Combi', 'Year'], columns=['Month'], 
                         values='Qty_Val', aggfunc=max)

df1_volume = df1_volume.reset_index().rename_axis(None).rename_axis(None, axis=1)

# Drop DataType and make it unique on 'Combi'
df1_volume = df1_volume.drop('DataType', axis=1)

df1_volume = df1_volume.drop_duplicates('Combi')

df1_volume = df1_volume.drop('Year', axis=1)

#...  Prepare the Cognos file ......
#...................................

# Merge forecast and plan
df_cognos = pd.concat([df1_forecast, df1_plan], ignore_index=True, sort=True)


# Merge with Volume file for details
df1_cognos = pd.merge(df_cognos, df1_volume, on='Combi')

# Split Combi into its constituents
df1_cognos = df1_cognos.assign(**df1_cognos.Combi.str.split
                       (' - ', expand=True).rename(columns = {0:'CustomerNo',
                                                              1:'ProductID', 
                                                              2:'Site_ID'}))

# Cognos quantities are in Millions. Change this to Units by multiplying by 1,000,000
df1_cognos['Qty'] = df1_cognos['Qty'] * 1000000

# Set By to the last date of the month
df1_cognos['By'] = (pd.to_datetime(df1_cognos.Year*10000+df1_cognos.MonthNum*100+1, format = "%Y%m%d") + \
                  pd.tseries.offsets.MonthEnd(1)).dt.date

# Map to the standardized Site
df1_cognos['Site'] = df1_cognos['Site_ID'].map(siteID_dict)

#...... Set to Essentra's accounting calendar ...
#................................................

# Some quarterly months are tricky in Essentra's accounting calendar. They are quarter end, 
# but donot align with Cognos!
change_qtr_last_day = {datetime.datetime(2018, 9, 30).date(): datetime.datetime(2018, 9, 29).date(),
                datetime.datetime(2019, 9, 30).date(): datetime.datetime(2019, 9, 28).date(),
				datetime.datetime(2019, 3, 31).date(): datetime.datetime(2019, 3, 30).date()}

df1_cognos.By = df1_cognos.By.replace(change_qtr_last_day)

df1_cognos = get_essby(df1_cognos)

# Cleanup columns and add new ones
df2_cognos = df1_cognos.drop(['Combi', 'Month', 'Year', 'MonthNum', 'Item ID', 'Site_ID'], axis=1)

df2_cognos = df2_cognos.rename(columns={"Prod Name": "ProductDesc",
                       "Cust Name": "CustName",
                       "Cust Country": "CustCountry"})
df2_cognos['Order'] = ''

df3_cognos = df2_cognos[base_cols+['EssBy']].reset_index(drop=True)

# Add Year_Priod and Period fields to Cognos
df3_cognos['Year_Period'] = df3_cognos.EssBy.astype(str).str.slice(0,7)
df3_cognos['Period'] = df3_cognos.Year_Period.str.slice(5,7).astype(int)

#*********************************************************************
#                            FourthShift Cleanup     
#_____________________________________________________________________

#... Cleanup the orders file ...
#...............................

df1_orders = df_orders.rename(columns = {'PromDock_InvoiceDate': 'By',
                          'CustomerCountry': 'CustCountry',
                          'CustomerName': 'CustName',
                          'Order_Invoice_No': 'Order',
                          'AccountingPeriod': 'Period'})

# Select only current year's data
df1_orders = df1_orders.loc[df1_orders.By >= start_date, :]

# Remove interco databy taking out CustName containing 'ESS', except for 'ALTESS' 
# and order containing CFG from Singapore

intercomask = df1_orders.CustName.str.contains('ESS', case=False) & \
              ~((df1_orders.CustName.str.contains('ALTESS', case=False)) | \
               (df1_orders.Order.str.contains('CFG', case=False)))

df2_orders = df1_orders[~intercomask].reset_index(drop=True)

# df2_orders = df1_orders.copy()

#... ItemUM cleanup
# Strip ItemUM of whitespace and nulls
df2_orders.ItemUM = df2_orders.ItemUM.str.strip()
df2_orders.loc[df2_orders.ItemUM.isnull(), "ItemUM"] = ''

# Replace MH and KR in ItemUm to TH
um_search = ['MH', 'KR']
df2_orders.loc[df2_orders.ItemUM.str.contains('|'.join(um_search), case=False), 'ItemUM'] = 'TH'

# Remove items with UoM = EA, but without RY, POP or VE in them. These are garbage (packaging and other data)
ryo_pop_search = ['RY', 'POP', 'VE']
df3_orders=df2_orders.loc[~(df2_orders.ItemUM.isin(['EA']) & 
                        ~(df2_orders.ProductID.str.contains('|'.join(ryo_pop_search), 
                                                            case=False, na=False))), :]

# Remove raw material orders - with ItemUM = KG, BB and CS
df4_orders = df3_orders[~(df3_orders.ItemUM.isin(['KG', 'BB', 'CS']))]

# Remove freight entries
df4_orders = df4_orders[df4_orders.ProductID.str.contains('FREIGHT', case=False) == False]

# Remove CPTT from Dubai
# df4_orders = df4_orders[~((df4_orders.Order.str.contains('CPTT', case=False) |
#                           df4_orders.ItemUM.str.contains('TH', case=False)) &
#                         (df4_orders.Site == 'AE'))]

# df4_orders.loc[(df4_orders.DataType == 'Order') &
#                (df4_orders.Period == 10) & 
#                (df4_orders.AccountingYear == 2018), 'GBPValue'].sum()

#.... RYO POP handling
#.....................

# Remove duplicates and description fields in df_ryo_pop
df_ryo_pop1 = df_ryo_pop.drop_duplicates(subset = 'Conv2TH').drop('Description', axis=1)

# Derive multiple's for ProductID: RYO_POP
df4_orders = pd.merge(df4_orders, df_ryo_pop1, left_on= 'ProductID',right_on='RYO_POP', how="left")

# For items that are 'RY' and 'POP', 
# ...with UoM as 'EA' and Conv2TH is a number, multiply it to Qty
# ...with UoM as 'EA' but Conv2TH is not a number, use conversion of 0.0125 * Qty
# ...change the UoM to 'TH'

# filter for RYO and POP only
df_ryo_pop_filter = ['RY', 'POP']

c1 = df4_orders.ProductID.str.contains('|'.join(df_ryo_pop_filter), case=False)
c2 = df4_orders.ItemUM.str.contains('EA', case=False)
c3 = pd.to_numeric(df4_orders.Conv2TH, errors='coerce').isnull()

# Multiply Conv2TH to Qty
df4_orders.loc[c1 & c2 & ~c3, 'Qty'] = pd.to_numeric(df4_orders.Qty) * pd.to_numeric(df4_orders.Conv2TH)

# Multiply Qty with 0.0125 (indicative) for null Conv2TH
df4_orders.loc[c1 & c2 & c3, 'Qty'] = pd.to_numeric(df4_orders.Qty) * 0.0125

# Change these UoM to 'TH'
df4_orders.loc[c1 & c2, 'ItemUM'] = 'TH'

# Convert all ItemUM = 'TH' to 'EA'
df4_orders.loc[df4_orders.ItemUM == 'TH', 'Qty'] = pd.to_numeric(df4_orders.Qty) * 1000
df4_orders.loc[df4_orders.ItemUM == 'TH', 'ItemUM'] = 'EA'

# Change all orders to OpenOrder
df4_orders.loc[df4_orders.DataType == 'Order', 'DataType'] = 'OpenOrder'

# Derive EssBy for orders

# df4_orders['EssBy'] = [k for x in df4_orders.By 
#                        for i, j, k in zip(df_periods.Start, df_periods.End, df_periods.EssBy) 
#                        if i <= x <= j]

df4_orders = get_essby(df4_orders)

#... Extract Brazil data (OpenOrders and Invoices) ...
#.....................................................

df_brazil = df_brazil.loc[df_brazil.By >= start_date, :]

df_brazil = df_brazil[['DataType', 'Site', 'CustomerNo', 'CustName', 'CustCountry', 'AM', 
                 'ProductID', 'ProductDesc', 'Order', 'By', 'Qty', 'GBPValue', 'StdGBPCost', 'Period']]

# Add UoM for conversion
df_brazil['ItemUM'] = 'TH'

# Extract Invoices and Copy them as Shipment
df_brazil_ship = df_brazil.loc[df_brazil.DataType == 'OpenOrder'].reset_index()
df_brazil_ship.loc[df_brazil_ship.DataType == 'Invoice', 'DataType'] = 'Shipment'

# Add Shipment to df_brazil
df_brazil = pd.concat([df_brazil, df_brazil_ship], axis=0, ignore_index=True, sort=True)

# convert all ItemUM = 'TH' to 'EA'
df_brazil.loc[df_brazil.ItemUM == 'TH', 'Qty'] = pd.to_numeric(df_brazil.Qty) * 1000

df_brazil = get_essby(df_brazil)

# Integrate orders with Brazil
df5_orders = pd.concat([df_brazil, df4_orders], sort=True)

#... Generate EssBy for df_f ..........
#......................................

df_f = pd.concat([df_f1, df_f2, df_f3, df_plan19], sort=True)

# Rename 'AM Code' column to 'AM'
df_f = df_f.rename(columns={'AM Code': 'AM'})

df_f.By = pd.to_datetime(df_f.By).dt.date

df_f.By = df_f.By.replace(change_qtr_last_day) # Adjusted the Septembers

# Replace dates of df_f before min(df_periods.Start) to df_periods.Start
df_f.loc[df_f.By < start_date, 'By'] = start_date

# Get EssBy and limit dataframe to minimum df_periods.Start
df_f = get_essby(df_f)

#... Generate EssBy for actuals ........
#.......................................

# For less than start_date push it to the first month.
df1_actuals = df_actuals.copy()
df1_actuals.loc[df1_actuals.By < start_date, 'By'] = df1_actuals.By.map(get_last_day) + \
                                                   datetime.timedelta(days=15)

# build Year_Period
df1_actuals_period = pd.to_datetime(df1_actuals.By).dt.strftime('%Y') \
                     +'-'+ df1_actuals.Period.map("{:02}".format)

# use dict to map EssBy
df1_actuals['EssBy'] = df1_actuals_period.map(df_essdict)

# Make the Actuals dataframe
df2_actuals = df1_actuals.copy()

df2_actuals.DataType = 'Actual'

#*********************************************************************
#                            'Demand' creation     
#_____________________________________________________________________

#...... Create EstSales ..............
#.....................................

# Get Shipment + OpenOrders for current month as EstSales
df0_estsales = df5_orders.loc[((df5_orders.DataType == 'Shipment') | 
                               (df5_orders.DataType == 'OpenOrder')) & 
                               (df5_orders.EssBy == thismonthend), :]

# Make future month forecasts as EstSales
df1_estsales = df3_cognos.loc[(df3_cognos.DataType == 'Forecast') & 
                              (df3_cognos.EssBy > thismonthend), :]

# Create EstSales

df_estsales = pd.concat([df0_estsales, df1_estsales], axis=0, sort=True).reset_index(drop=True)
df_estsales.loc[:, 'DataType'] = 'EstSales'

#... Consolidate cognos, f1, f2, f3, actuals, estsales ...
#................................................

base_cols1 = base_cols + ['EssBy', 'Period']

df0_demand = pd.concat([df3_cognos[base_cols1], df_f[base_cols1], df1_actuals[base_cols1],
                        df2_actuals[base_cols1], df_estsales[base_cols1]]).reset_index(drop=True)

df1_demand = pd.concat([df0_demand[base_cols1], df5_orders[base_cols1]], axis=0, sort=True)

# Add manufacturing site
df1_demand['MfgSite'] = np.where((df1_demand["Site"] == 'SG') & 
                                 (df1_demand.CustomerNo.str.slice(4,5) == 'T'), 'TH', df1_demand.Site)
df1_demand['MfgSite'] = np.where((df1_demand["Site"] == 'SG') & 
                                 (df1_demand.CustomerNo.str.slice(4,5) != 'T'), 'ID', df1_demand.MfgSite)
df1_demand['MfgSite'] = np.where((df1_demand["Site"] == 'BR'), 'PA', df1_demand.MfgSite)

# Split SG to TH and ID. Consolidate BR into PA.
df1_demand.loc[(df1_demand["Site"] == 'SG') & (df1_demand.CustomerNo.str.slice(4,5) == 'T'), "Site"] = 'TH'
df1_demand.loc[(df1_demand["Site"] == 'SG') & (df1_demand.CustomerNo.str.slice(4,5) != 'T'), "Site"] = 'ID'
df1_demand.loc[(df1_demand["Site"] == 'BR'), "Site"] = 'PA'

# Manufacturing site change - based on special routing 
df_mfgsite = df_mfgsitechange[['CustomerNo', 'ProductID', 'Site', 'MfgSite']]
mfgsite_dict = df_mfgsite.set_index(['CustomerNo', 'ProductID', 'Site']).to_dict(orient='index')
mfgsite_dict = {(k[0], k[1], k[2]): vm for k, v in mfgsite_dict.items() for km, vm in v.items()}

l = [i for i in zip(df1_demand.CustomerNo, df1_demand.ProductID, df1_demand.Site)]

def catch(func, handle=lambda e : e, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except Exception as e:
        return np.nan

m = [catch(lambda: mfgsite_dict[m]) for m in l]
df1_demand['MfgSite'] = m

# Replace MfgSite having nan with Site
df1_demand.loc[~df1_demand.MfgSite.notnull(), 
               'MfgSite'] = df1_demand.loc[~df1_demand.MfgSite.notnull(), 'Site']

#.....   Handle no current month orders in India .....
#.....................................................

if no_india_orders:
    
    # Demand without EstSales in India for the current period
    df2_demand = df1_demand.loc[~((df1_demand.DataType == 'EstSales') & 
                       (df1_demand.Site == 'IN') & 
                       (df1_demand.EssBy == thismonthend)), :]

    # Demand with Forecast in India for current period (to replace EstSales)
    df3_demand = df2_demand.loc[((df2_demand.DataType == 'Forecast') & 
                       (df2_demand.Site == 'IN') & 
                       (df2_demand.EssBy == thismonthend)), :].reset_index(drop=True)

    df3_demand.loc[:, 'DataType'] = 'EstSales'

    # Concatenate with remaining demand
    df2_demand = pd.concat([df2_demand, df3_demand]).reset_index(drop=True)
else:
    df2_demand = df1_demand.copy()

#.....   Handle no actuals for last month  ......
#................................................

last_month_EssBy = get_last_day(datetime.datetime.now().replace(day=15).date()
                                -datetime.timedelta(days=17))

no_actuals = df1_actuals.By.max() < last_month_EssBy

if no_actuals:
    df_tmp_actuals = df2_demand.loc[(df2_demand.EssBy == last_month_EssBy) & 
                   ((df2_demand.DataType == 'OpenOrder') |
                   (df2_demand.DataType == 'Shipment')) , :].reset_index(drop=True)
    
    df_tmp_actuals.loc[:, 'DataType'] = 'EstSales'
    
    df2_demand = pd.concat([df2_demand, df_tmp_actuals]).reset_index(drop=True)

#*********         Fillups and Lookups    ****************
#______________________________________________________

df3_demand = df2_demand.copy()

# Fill up the TLAs

df_prodref = pd.read_excel(lookups_xls, 'ProductRef', header=0, skiprows=4)
df_prodref = df_prodref.rename(columns = {'S1': 'Seg1', 
                                            'S2': 'Seg2', 'S3': 'Seg3', 'S4': 'Seg4' })
df_prodref = df_prodref.fillna('')       # Replace all NaN with blanks
df3_demand['TLA'] = df3_demand.ProductID.str.slice(0,3)

df3_demand = pd.merge(df3_demand, df_prodref, left_on='TLA', 
                             right_on='TLA', how='left')

#.....        CustomerGrp and CustomerType Lookup      ......
#............................................................
df_custgrp = df_custgrp.drop_duplicates('CustName')

df_custgrp1 = df_custgrp[['CustName', 'CustomerGrp', 'Region', 'CustomerType']]

df3_demand = pd.merge(df3_demand, df_custgrp1, on = 'CustName', how = 'left')

# Replace missing CustCountry from looked-up CustName -> CustCountry fields
df_custcountry = df_custgrp[['CustName', 'CustCountry']].set_index('CustName')
mask_cc = df3_demand.CustCountry.isnull()
df3_demand.loc[mask_cc, 'CustCountry'] = df3_demand.loc[mask_cc, 
                                                    :].CustName.map(df_custcountry.CustCountry)

#.....                        AM Lookup                ........
#..............................................................

df_am = df_am.drop_duplicates('AM Code')

df3_demand = pd.merge(df3_demand, df_am, left_on= "AM", right_on='AM Code', 
         how='left').drop(['AM', 'AM Code'], 
                          axis=1).rename(columns={"AM Name": "AM"})

# Replace missing AMs from looked-up CustName -> AM fields
df_custAM = df_custgrp[['CustName', 'AM Name']].set_index('CustName')
mask_am = df3_demand.AM.isnull() & df3_demand.CustName.notnull()
df3_demand.loc[mask_am, 'AM'] = df3_demand.loc[mask_am, :].CustName.map(df_custAM['AM Name'])

#... Identifying Game Changers ...
#.................................
df_gamechangers = pd.read_excel(lookups_xls, 'GameChangers', header=0, skiprows=4, usecols=[1,2])

df_gc_dict = {df['Item']: df['GameC'] for df in df_gamechangers.to_dict(orient='records')}

df3_demand['GameC'] = df3_demand.TLA.map(df_gc_dict)
df3_demand.GameC = df3_demand.ProductID.map(df_gc_dict).fillna(df3_demand.GameC)

# If there is any demand in month 13, replace it with 'By' period's extract
df3_demand.loc[(df3_demand.Period == 13), 'Period'] = pd.to_datetime(df3_demand.loc[(df3_demand.Period == 13), 'By']).dt.month

##*********************************************************************
#                            'Supply' creation     
#_____________________________________________________________________

# Keep only Order, Forecast and EstSales in demand
demand = df3_demand.loc[(df3_demand['DataType'] == 'OpenOrder') | 
                        (df3_demand['DataType'] == 'EstSales') |
                        (df3_demand['DataType'] == 'Forecast')]
demand = demand.reset_index(drop=True)

#... Determine appropriate manufacturing site
#............................................

# For Singapore and Brazil
demand.loc[(demand["Site"] == 'SG') & (demand.CustomerNo.str.slice(4,5) == 'T'), "Site"] = 'TH'
demand.loc[(demand["Site"] == 'SG') & (demand.CustomerNo.str.slice(4,5) != 'T'), "Site"] = 'ID'
demand.loc[(demand["Site"] == 'BR'), "Site"] = 'PA'

# For out-of-self manufacturing sites (like Dubai)
mfg_site = df_mfgsitechange[["CustomerNo", "ProductID", "MfgSite"]]

mfg_site = mfg_site.sort_values(by= ["CustomerNo", "ProductID"])
demand = demand.sort_values(by=['CustomerNo', 'ProductID'])

mfg_site = mfg_site.set_index(['CustomerNo', 'ProductID'])
demand = demand.set_index(['CustomerNo', 'ProductID'])
demand.update(mfg_site) # Update MfgSite column

mfg_site1 = df_mfgsitechange[["CustomerNo", "ProductID", "MfgSite"]]
mfg_site1.columns = ["CustomerNo", "ProductID", "Site"]
mfg_site1 = mfg_site1.set_index(['CustomerNo', 'ProductID'])
demand.update(mfg_site1) # Update Site column to be the same as MfgSite

demand = demand.reset_index()
demand = demand.drop('BRMult', axis=1)

#... Integrate with BOM
# Weed out the BOM of unnecessary rows
bom = df_bom.loc[df_bom.ComponentItemNumber.str.match('...(\d)') &
        df_bom.ComponentItemNumber.str.match('..([A-Z])') &
        (df_bom.ComponentItemNumber.str[:3] != 'CON') &
        (df_bom.RequiredQuantity != 0) &
        (df_bom.ComponentType.str[0] == 'N') &
        (pd.to_datetime(df_bom.OutEffectivityDate) >= 
            datetime.datetime.now()), :].reset_index(drop=True)

machine_cols = ["Site", "Machine", "MachineID", "MachineType", 
                     "Category", "Size", "Spec1", "Spec2", "Spec3",
                     "DemoCap_ph", "MaxCap_ph", 
                     "Shift", "OfflineFrom", "OfflineTo"]

# Add a column to demand and bom with 'SiteID/ProductIDmachine_cols'
demand["SiteProduct"] = demand["Site"] + '/' + demand["ProductID"]
bom["SiteProduct"] = bom["Site"] + '/' +  bom["ItemNumber"]

# ...Convert Forecast to EstSales for India for demand
#    This is to prevent the India demand shortfall to supply during the first 10 days

# remove EstSales for India in this month
demand = demand[~((demand.Site == 'IN') & 
       (demand.DataType == 'EstSales') &
       (demand.EssBy == thismonthend))]

# Fish out the current month's forecast for India and call it to be EstSales
in_curr_estsales = demand[((demand.Site == 'IN') & 
       (demand.DataType == 'Forecast') &
       (demand.EssBy == thismonthend))]

in_curr_estsales = in_curr_estsales.assign(DataType = 'EstSales')

# append India's estimated sales to demand
demand = demand.append(in_curr_estsales).reset_index(drop=True)

# remove Forecast from demand. Everything below should be based on EstSales
demand = demand[demand.DataType != 'Forecast']

####------------------------------------------------------------------------------------
#        PREPARING BASEROD DEMAND
####------------------------------------------------------------------------------------

# Merge demand with bom to get baserod demand
cols_to_drop = ['SiteProduct', 'ItemNumber', 'Site_y']
baserod = pd.merge(demand, bom, how='inner', on='SiteProduct').drop(cols_to_drop, 1)   \
    .rename(columns={'ComponentItemNumber': 'BaseRod', 'RequiredQuantity':'BRMult', 'Site_x': 'Site'})

    
baserod.loc[(baserod.DataType == "OpenOrder") & 
           ((baserod.Category == "Dual") | (baserod.Category == "Triple") | 
           (baserod.Category == "Quad")),
           "DataType"] = 'OrdrBR'

baserod.loc[(baserod.DataType == "EstSales") & 
           ((baserod.Category == "Dual") | (baserod.Category == "Triple") | 
           (baserod.Category == "Quad")),
           "DataType"] = 'FcstBR'

baserod.loc[~((baserod.DataType == "OrdrBR") | (baserod.DataType == "FcstBR")) &
            (baserod.DataType == "OpenOrder"),
            "DataType"] = 'OrdrOth'

baserod.loc[~((baserod.DataType == "OrdrBR") | (baserod.DataType == "FcstBR")) &
            (baserod.DataType == "EstSales"),
            "DataType"] = 'FcstOth'

# Add baserod to ProductID
# baserod.ProductID = baserod.BaseRod + '->' +baserod.ProductID
baserod.rename(columns={"ProductID": "Parent", "BaseRod": "ProductID"}, inplace=True)

# Change category for Dual, Triple and Quad to Mono.
# This is to match against Mono capacity.

baserod.loc[(baserod.Category == "Dual") | (baserod.Category == "Triple") | (baserod.Category == "Quad"),
             "Category"] = "Mono"

# # Add standalone baserrod sku to baserod demand
mono_sku = demand.loc[((demand.DataType == "OpenOrder") | (demand.DataType == "EstSales")) &(demand.Category == "Mono")]

# Make the DataType as FcstBR
mono_sku = mono_sku.reset_index(drop=True)
mono_sku.loc[(mono_sku.DataType == "OpenOrder"), "DataType"] = "OrdrBR"
mono_sku.loc[(mono_sku.DataType == "EstSales"), "DataType"] = "FcstBR"

# Add BRMult column with 1 to mono_sku
mono_sku.insert(loc=8, column="BRMult",value=1)

# baserod['SiteProduct'] = ''
# mono_sku['Parent'] = ''

baserod = pd.concat((baserod, mono_sku), axis=0, ignore_index=True, sort=True)

baserod.Qty = baserod.Qty * baserod.BRMult

baserod = baserod[baserod.DataType.isin(['FcstBR', 'OrdrBR'])]

####------------------------------------------------------------------------------------
#        PREPARING COMBINER DEMAND
####------------------------------------------------------------------------------------

# For Combiner Order
demand.loc[(demand.DataType == "OpenOrder") & 
          ((demand.Category == "Dual") | (demand.Category == "Triple") | (demand.Category == "Quad")),
           "DataType"] = 'OrderComb'

# For Combiner Forecast
demand.loc[(demand.DataType == "EstSales") & 
          ((demand.Category == "Dual") | (demand.Category == "Triple") | (demand.Category == "Quad")),
           "DataType"] = 'FcstComb'

# drop SiteProduct
demand = demand.drop("SiteProduct", 1)

# keep only combiner demand
demand = demand[demand.DataType.isin(["FcstComb", "OrderComb"])]

####------------------------------------------------------------------------------------
#        PREPARE OVERALL DEMAND PLAN (BASEROD + COMBINER)
####------------------------------------------------------------------------------------

demand_plan = pd.concat([demand, baserod], sort=True)

####------------------------------------------------------------------------------------
#        SPLIT MONTHLY DEMAND TO WEEKLY BUCKETS
####------------------------------------------------------------------------------------

# ## Set the dates

first_day = df_periods.Start.min()

# Set today's date as the first day of the month
if date.today().month != 1:
    today = date(date.today().year, date.today().month-1,1)
else:
    today = date(date.today().year, 1, 1)

# Function to get the date of the previous Weekday before the given date
# Monday will be 0
onDay = lambda date, day: date - datetime.timedelta(days=(day+date.weekday()+7)%7)

today = onDay(today,0) # Set today to the first Monday of the Month, even if it is in previous month

# Set last day based on SnOP timeframe of 24 months
last_day = today + pd.DateOffset(months=24)
last_day = pd.to_datetime(last_day).date()

# Generate date list of Mondays - from today till the last day
mondays = pd.Series(pd.date_range(today, last_day, freq='W-Mon'))

weeks = pd.DataFrame({'Week':mondays})

# Remove dates greater than first_day and less than last_day from demand_plan
demand_plan = demand_plan[(demand_plan.By > first_day) & (demand_plan.By < last_day )]

####------------------------------------------------------------------------------------
#        GENERATE CAPACITIES
####------------------------------------------------------------------------------------

machine = df_machine[machine_cols]

# Strip all texts in machine
machine_obj = machine.select_dtypes(['object'])
machine[machine_obj.columns] = machine_obj.apply(lambda x: x.str.strip())

# Generate WeekFrom and WeekTo records for each date in the DataFrame
machine = machine.assign(key=0)

ser = pd.DataFrame({'WeekFrom': mondays, 'key':[0] * len(mondays)}) # generate dataframe series with key

machine = pd.merge(machine, ser, on = 'key').drop('key', axis = 1)

machine['WeekTo'] = machine['WeekFrom'].apply(lambda x: x + timedelta(days=7))

# Convert all mondays to datetime for comparison
machine['WeekFrom'] = pd.to_datetime(machine['WeekFrom'], errors='coerce')
machine['WeekTo'] = pd.to_datetime(machine['WeekTo'], errors='coerce')
machine['OfflineFrom'] = pd.to_datetime(machine['OfflineFrom'], errors='coerce')
machine['OfflineTo'] = pd.to_datetime(machine['OfflineTo'], errors='coerce')

## Extract number of days per week from 'Shift' column.
## Subtract holidays from 'Shift' column, if the holiday:
##   - it falls within 'WeekFrom' and 'WeekTo', 
##  - and is not within 'OfflineFrom' and 'OfflineTo' 

# Get the days from 'Shift' and convert it to a numeric
machine['ShiftDays'] = pd.to_numeric(machine['Shift'].str[3:])

# Get the hours from 'Shift' and convert it to a numeric
machine['ShiftHours'] = pd.to_numeric(machine['Shift'].str.slice(0,2))

# Get the holidays
df_holidays.dropna(how='all', inplace=True)
holiday = df_holidays

# Convert holiday array's values to datetime for comparison
holiday['Holiday'] = pd.to_datetime(holiday['Holiday'])

# Reduce ShiftDays by number of holidays in each site for each machine line.
machine.ShiftDays -= ((holiday.Holiday[:, None] >= machine.WeekFrom.values)
                   & (holiday.Holiday[:, None] <= machine.WeekTo.values) 
                   & (holiday.SiteID[:, None] == machine.Site.values)).sum(axis=0)

## Condition table for Online Days:
# Initialize Days column
machine['Days'] = machine['ShiftDays']

#-----------------------------------------------------------------------------------------------------
#                                  ---OfflineFrom---OfflineTo---
#         WeekFrom WeekTo----------|                            |-----WeekFrom WeekTo

# ........There is nothing to do here as the default for these is ShiftDays.........
# machine.loc[(((machine.WeekFrom <= machine.OfflineFrom) & (machine.WeekTo <= machine.OfflineFrom)) | 
#             ((machine.WeekFrom >= machine.OfflineTo) & (machine.WeekTo >= machine.OfflineTo))), :]
# ....................................................................................................

# ..........................     First condition       ...............................................
#           OfflineFrom|-----------------------------------------------------|OfflineTo
#                             WeekFrom | -------------------------|  Week To

# Set mask1 to locate the condition
mask1 = (machine.OfflineFrom <= machine.WeekFrom) & (machine.OfflineTo >= machine.WeekTo)
machine.loc[mask1, "Days"] = 0

# .......................        Second condition      .......................     
#                             ---OfflineFrom-----------------OfflineTo---
#         WeekFrom -----------|                WeekTo
# ............................................................................

# Set mask2 to locate the condition
mask2 = ((machine.WeekFrom <= machine.OfflineFrom) & (machine.WeekTo <= machine.OfflineTo) & 
         (machine.WeekTo >= machine.OfflineFrom))

# Compute the days for the mask2 based on the condition.        
dayseries = machine.loc[mask2, "ShiftDays"] - \
           (machine.loc[mask2, "WeekTo"] - machine.loc[mask2, "OfflineFrom"]).dt.days

# Set the Days and remove any negative days with max function
machine.loc[mask2, "Days"] = np.maximum(dayseries, 0)

# .......................       Third condition   ................................     
#                             ---OfflineFrom-------------OfflineTo---|
#                                          WeekFrom |-------------------WeekTo-----|
# ................................................................................

# Set mask3 to locate the condition
mask3 = (machine.OfflineFrom < machine.WeekFrom) & \
        (machine.WeekFrom < machine.OfflineTo) & \
        (machine.OfflineTo > machine.WeekTo)
     
# Compute the days for the mask3 based on the condition
dayseries3 = (machine.loc[mask3, "WeekTo"] - machine.loc[mask3, "OfflineTo"]).dt.days

# Set the Days to minumum of ShiftDays and dayseries3
machine.loc[mask3, "Days"] = np.minimum(machine.loc[mask3, "ShiftDays"], dayseries3)

# Remove negative Days from machines (could happen if entire week was a holiday)
machine.loc[machine.Days < 0, "Days"] = 0

# Determine Demonstrated machine and Maximum machine
machine['DemoCap'] = machine['DemoCap_ph'] * machine['ShiftHours'] * machine['Days'] * 1000

machine['MaxCap'] = machine['MaxCap_ph'] * 24 * 7 * 1000

# Rename 'WeekTo' to 'By' and 'DemoCap' to 'Qty'
machine = machine.rename(columns={'WeekTo': 'By', 'DemoCap': 'Qty', 'MachineType': 'DataType'})

####------------------------------------------------------------------------------------
#        CREATE THE SUPPLY PLAN
####------------------------------------------------------------------------------------

#... Prepare supply_plan from demand_plan
supply_plan = demand_plan.copy()

# Remove out-of-range period data
supply_plan = supply_plan[~(supply_plan.By > df_periods.End.max())].reset_index(drop=True)

# Generate EssBy for Essentra calendar
supply_plan['EssBy'] = [k for x in supply_plan.By
                       for i, j, k in zip(df_periods.Start, df_periods.End, df_periods.EssBy) 
                       if i <= x <= j]

# Add Year_Period and Period to the supply plan
supply_plan['Year_Period'] = supply_plan.EssBy.astype(str).str.slice(0,7)
supply_plan['Period'] = supply_plan.Year_Period.str.slice(5,7).astype(int)

# Stage the machine_map for supply
machine_map_cols = ['MachineID', 'Site', 'Year_Period', 'TLA', 'ProductID']
machine_map = df_machinemap[machine_map_cols].drop_duplicates()

# Get the MachineID column in supply_plan for TLA
m = machine_map[['Site', 'Year_Period', 'TLA', 'MachineID']]
supply_plan = supply_plan.merge(m.drop_duplicates(['Site', 'Year_Period', 'TLA']), how='left')

#... Update the MachineID column in supply_plan for ProductID
s = supply_plan.set_index(['Site', 'Year_Period', 'ProductID'])
mp = machine_map[['Site', 'Year_Period', 'ProductID', 'MachineID']].drop_duplicates(['Site', 'Year_Period', 'ProductID']).set_index(['Site', 'Year_Period', 'ProductID'])

s.update(mp, overwrite=False)

supply_plan = s.reset_index()

# merge supply_plan with compacted machine, to get additional machine fields
cm = df_machine[['Site', 'MachineID', 'Machine', 'MaxCap_ph']].drop_duplicates()
cm = cm.assign(MaxCap= cm.MaxCap_ph*7*24*1000).drop('MaxCap_ph', axis=1) # computed MaxCap

supply_plan = supply_plan.merge(cm, how='left')

#... Prepare the machine data
# Remove out-of-range period data
machine.By = pd.to_datetime(machine.By).dt.date # convert to date
machine = machine[~(machine.By > df_periods.End.max())].reset_index(drop=True)

machine['EssBy'] = [k for x in machine.By
                       for i, j, k in zip(df_periods.Start, df_periods.End, df_periods.EssBy) 
                       if i <= x <= j]

# Add Year_Period and Period to the machines
machine['Year_Period'] = machine.EssBy.astype(str).str.slice(0,7)
machine['Period'] = machine.Year_Period.str.slice(5,7).astype(int)

# merge the machines with machinemap to get TLA and ProductID of the machines
machine = machine.merge(machine_map.drop_duplicates(['MachineID', 'Site', 'Year_Period']), on=['MachineID', 'Site', 'Year_Period'], how='left')

# Make a clean product reference table to update machine for deltails through TLA
df_prodref1 = df_prodref[['TLA', 'Seg1', 'Seg2', 'Seg3', 'Seg4', 'Feature', 'SubFeature']].drop_duplicates()
machine = pd.merge(machine, df_prodref1, on='TLA', how='left' )

# concatenate supply_plan and machine to make supply
supply = pd.concat((supply_plan, machine), axis=0, ignore_index=True, sort=True)

# For ProductID with Parent, replace the TLA to be of ProductID and not of Parent
supply.loc[supply.Parent.notnull(), 'TLA'] = supply.ProductID.str.slice(0,3)

# remove 'Other' category from Supply. These are all e-Cigs
supply = supply[supply.DataType != 'Other']

####------------------------------------------------------------------------------------
#        HANDLING EXCEPTIONS
####------------------------------------------------------------------------------------

executed = [exec(i) for i in df_exceptions.Logic]

#**********************************************************************
#                       Costing distribution
#_______________________________________________________________________

#......  Create UnitCost dictionaries ............
#.................................................

def get_avg_unit_cost(df, u):
    '''Args: 
         df as pandas.DataFrame, u as unitcost column header
       Returns:
         dictionary of Site+ProductID and Average Unit Cost'''
    df1 = df[['Site', 'ProductID', u]].dropna(subset=[u]).reset_index(drop=True)
    df1['Product9'] = df1.Site + df1.ProductID.str.slice(0,9)
    df1 = df1[['Product9', u]].set_index('Product9') # 9 digit ProductID
    df1_dict = df1.groupby('Product9').mean().to_dict(orient='index')
    thedict = {k: v1 for k, v in df1_dict.items() for k1, v1 in v.items()}
    return thedict

# Get UnitCost from Actuals
df_actuals['UnitCost'] = df_actuals.StdGBPCost/df_actuals.Qty
df_actuals['UnitLabourCost'] = df_actuals.Labour/df_actuals.Qty
df_actuals['UnitMaterialCost'] = df_actuals.Material/df_actuals.Qty
df_actuals['UnitFixedOH'] = df_actuals.FixedOH/df_actuals.Qty
df_actuals['UnitVarOH'] = df_actuals.VarOH/df_actuals.Qty

# df_actuals['UnitCost'] = df_actuals.StdGBPCost/df_actuals.Qty * np.where(df_actuals.GBPValue < 0, -1, 1)
# df_actuals['UnitLabourCost'] = df_actuals.Labour/df_actuals.Qty * np.where(df_actuals.GBPValue < 0, -1, 1)
# df_actuals['UnitMaterialCost'] = df_actuals.Material/df_actuals.Qty * np.where(df_actuals.GBPValue < 0, -1, 1)
# df_actuals['UnitFixedOH'] = df_actuals.FixedOH/df_actuals.Qty * np.where(df_actuals.GBPValue < 0, -1, 1)
# df_actuals['UnitVarOH'] = df_actuals.VarOH/df_actuals.Qty * np.where(df_actuals.GBPValue < 0, -1, 1)

avg_actual_unit_cost_dict = get_avg_unit_cost(df_actuals, 'UnitCost')
avg_actual_unit_labour_cost_dict = get_avg_unit_cost(df_actuals, 'UnitLabourCost')
avg_actual_unit_material_cost_dict = get_avg_unit_cost(df_actuals, 'UnitMaterialCost')
avg_actual_unit_fixedoh_cost_dict = get_avg_unit_cost(df_actuals, 'UnitFixedOH')
avg_actual_unit_varoh_cost_dict = get_avg_unit_cost(df_actuals, 'UnitVarOH')

# Get UnitCost from Orders
df5_orders['UnitCost'] = df5_orders.StdGBPCost/df5_orders.Qty
df5_orders['UnitLabourCost'] = df5_orders.Labour/df5_orders.Qty
df5_orders['UnitMaterialCost'] = df5_orders.Material/df5_orders.Qty
df5_orders['UnitFixedOH'] = df5_orders.FixedOH/df5_orders.Qty
df5_orders['UnitVarOH'] = df5_orders.VarOH/df5_orders.Qty

# df5_orders['UnitCost'] = df5_orders.StdGBPCost/df5_orders.Qty * np.where(df5_orders.GBPValue < 0, -1, 1)
# df5_orders['UnitLabourCost'] = df5_orders.Labour/df5_orders.Qty * np.where(df5_orders.GBPValue < 0, -1, 1)
# df5_orders['UnitMaterialCost'] = df5_orders.Material/df5_orders.Qty * np.where(df5_orders.GBPValue < 0, -1, 1)
# df5_orders['UnitFixedOH'] = df5_orders.FixedOH/df5_orders.Qty * np.where(df5_orders.GBPValue < 0, -1, 1)
# df5_orders['UnitVarOH'] = df5_orders.VarOH/df5_orders.Qty * np.where(df5_orders.GBPValue < 0, -1, 1)

avg_order_unit_cost_dict = get_avg_unit_cost(df5_orders, 'UnitCost')
avg_order_unit_labour_cost_dict = get_avg_unit_cost(df5_orders, 'UnitLabourCost')
avg_order_unit_material_cost_dict = get_avg_unit_cost(df5_orders, 'UnitMaterialCost')
avg_order_unit_fixedoh_cost_dict = get_avg_unit_cost(df5_orders, 'UnitFixedOH')
avg_order_unit_varoh_cost_dict = get_avg_unit_cost(df5_orders, 'UnitVarOH')

# Merge the Orders dictionary with Actuals (stackoverflow.com/a/26853961/7978112)
x = avg_order_unit_cost_dict
y = avg_actual_unit_cost_dict

def merge_two_dicts(x, y):
    '''Merges two dictionaries
    Args:
       (x, y) as dict objects
    Returns:
       y replacing values in x as a dict'''
    z = x.copy() # start with x's keys and values
    z.update(y)  # modifies z with y's keys and values & returns None
    return z

unitcost_dict = merge_two_dicts(avg_order_unit_cost_dict, avg_actual_unit_cost_dict)
labour_dict = merge_two_dicts(avg_order_unit_labour_cost_dict, avg_actual_unit_labour_cost_dict)
material_dict = merge_two_dicts(avg_order_unit_material_cost_dict, avg_actual_unit_material_cost_dict)
fixedoh_dict = merge_two_dicts(avg_order_unit_fixedoh_cost_dict, avg_actual_unit_fixedoh_cost_dict)
varoh_dict = merge_two_dicts(avg_order_unit_varoh_cost_dict, avg_actual_unit_varoh_cost_dict)

# Make UnitPrice and Margin columns and replace infinities with nan
df5_orders['UnitPrice'] = df5_orders.GBPValue / df5_orders.Qty
df5_orders ['StdMargin'] = (df5_orders.UnitPrice - df5_orders.UnitCost)/df5_orders.UnitPrice
df5_orders = df5_orders.replace([np.inf, -np.inf], np.nan)  # Replace infinities with nan

# Make a Site+/TLA unitcost dictionaries
a = pd.DataFrame.from_dict({k[0:5]: v for k, v in unitcost_dict.items()}, orient='index', columns=['UnitCost'])
df_sitetlacost = a.reset_index().groupby('index').mean().reset_index()
sitetlacost_dict = df_sitetlacost.to_dict(orient='index')
sitetlacost_dict = {v['index']: v['UnitCost'] for k, v in sitetlacost_dict.items()}

margin_fields = 'UnitCost,UnitLabourCost,UnitMaterialCost,UnitFixedOH,UnitVarOH,UnitPrice,StdMargin'.split(',')

def get_unit_cost(df):
    '''Extends dataframe to include unit cost columns
    Arg: df as DataFrame
    Returns: extended df'''
    siteproduct = df.Site+df.ProductID.str.slice(0,9)

    # Prepare the demand columns
    df['UnitCost'] = siteproduct.map(unitcost_dict)
    df['UnitLabourCost'] = siteproduct.map(labour_dict)
    df['UnitMaterialCost'] = siteproduct.map(material_dict)
    df['UnitFixedOH'] = siteproduct.map(fixedoh_dict)
    df['UnitVarOH'] = siteproduct.map(varoh_dict)
    
    sitetla = df.Site+df.TLA
    df['UnitCost'] = sitetla.map(sitetlacost_dict)
    df.loc[df.UnitCost.isnull(), 'UnitCost'] = sitetla.map(sitetlacost_dict)

#     df_wo_UnitCost = df[pd.to_numeric(df.UnitCost, errors='coerce').isnull()]

    # Make Qty 1 for removing na errors
#     df.loc[(df.DataType == 'EstSales') & (df.Qty == 0.0), 'Qty'] = 1
    df.loc[(df.Qty == 0.0), 'Qty'] = 1

    # Make UnitPrice, StdMargin, StdGBPCost and Period columns
    df['UnitPrice'] = df.GBPValue / df.Qty
    df['StdMargin'] = (df.UnitPrice - df.UnitCost).div(df.UnitPrice.where(df.UnitPrice !=0, np.nan))
    df['StdGBPCost'] = df.UnitCost * df.Qty
    df['Year_Period'] = df.EssBy.map({v:k for k, v in df_essdict.items()})

    # .... Handle margin outliers  ......

    # Remove costs for ProductIDs having SAM in it.
    df.loc[df.ProductID.str.upper().str.contains('SAM', na=False), 
                 margin_fields] = pd.np.nan

    if remove_outliers:
        mask = (df.StdMargin < negative_outlier) | (df.StdMargin > positive_outlier)
        df.loc[mask, margin_fields] = df.loc[mask, margin_fields].assign(StdMargin=np.nan)

    df = df.replace([np.inf, -np.inf], np.nan)  # Replace infinities with nan

    return(df)

# Get unit costs for demand and supply
df_demand = get_unit_cost(df3_demand)

df_supply = get_unit_cost(supply)

####      Output csvs to file    #####
#________________________________

df_demand_cols = 'DataType,Site,CustomerNo,CustName,CustCountry,AM,ProductID,ProductDesc,\
Order,By,Qty,GBPValue,StdGBPCost,CustomerGrp,CustomerType,Region,Category,SubCat,\
Feature,SubFeature,Size,Period,MfgSite,TLA,GameC'.split(',')

df_demand.to_csv(localpath+'\_c4s.csv', columns = df_demand_cols, index=False)

df_supply_cols_final = ["DataType","Site","CustomerNo","CustName","CustCountry","AM",
                     "ProductID","ProductDesc","Parent","Order","By","Qty","GBPValue",
                     "StdGBPCost","CustomerGrp","CustomerType","Region","Size","Category",
                     "MachineID","Machine","MaxCap","Seg1","Seg2","Seg3","Seg4",
                     "Spec1","Spec2","Spec3","Feature","Year_Period","TLA"]

df_supply.to_csv(path_or_buf = localpath+'\_supply.csv', columns = df_supply_cols_final, index  = False)

# Extract data for margin analysis
df_margin = df_demand[~df_demand.StdGBPCost.isnull() & 
                      (df_demand.DataType == 'EstSales') & 
                      (df_demand.TLA.isin(list(df_prodref1.TLA)))].reset_index(drop=True)

# df_margin.to_csv(path_or_buf = localpath+'\_margin.csv', columns = df_demand_cols, index  = False)


# For margin details extraction
margin_cols = df_demand_cols + ['UnitLabourCost', 'UnitMaterialCost', 'UnitFixedOH', 'UnitVarOH']

df_mgn = df_margin[margin_cols]
df_mgn = df_mgn.assign(Labour = df_mgn.UnitLabourCost*df_mgn.Qty)
df_mgn = df_mgn.assign(Material = df_mgn.UnitMaterialCost*df_mgn.Qty)
df_mgn = df_mgn.assign(FixedOH = df_mgn.UnitFixedOH*df_mgn.Qty)
df_mgn = df_mgn.assign(VarOH = df_mgn.UnitVarOH*df_mgn.Qty)

mgn_cols = df_demand_cols + ['Labour', 'Material', 'FixedOH', 'VarOH']
# df_mgn.to_csv(path_or_buf = localpath+'\_margin_details.csv', columns = mgn_cols, index  = False)

end = time.time()

print(f"Successfully completed the program in {(end-start)/60} minutes")

In [ ]:
#...Check for India's EstSales for current period
df_demand[(df_demand.Site == 'IN') & 
          (df_demand.EssBy == thismonthend) &
          (df_demand.DataType == 'EstSales')].GBPValue.sum()

In [ ]:
df5_orders[(df5_orders.Site == 'IN') & (df5_orders.EssBy == datetime.date(2019, 3, 31))].GBPValue.sum()

In [ ]:
thismonthend